In [16]:
import pandas as pd
import requests
import openai

openai.api_key = "{API KEY}"

def collectmovies(params):
    # Construct the URL
    url = "https://api.themoviedb.org/3/discover/movie?" + "&".join([f"{key}={value}" for key, value in params.items()])
    headers = {"accept": "application/json","Authorization": "BEARER AUTHORIZATION CODE"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:

    data = response.json()
    movies = data.get("results", [])
    df = pd.DataFrame(movies)

    else:
    print("Error fetching data. Status code:", response.status_code)

    df.drop(columns=["backdrop_path","id","poster_path","title","video","adult","original_language"], inplace=True)
    urls = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    response = requests.get(urls, headers=headers)
    genres_data = response.json()
    genre_map = {genre["id"]: genre["name"] for genre in genres_data.get("genres", [])}
    df["genre_names"] = df["genre_ids"].apply(lambda ids: [genre_map[id] for id in ids])
    df.drop(columns=["genre_ids"], inplace=True)
    return df



def movie_list(df, genre):
    filtered_list = df[df['genre_names'].apply(lambda x: genre in x)]

    if not filtered_list.empty:
        # print(f"Recommended {genre} movies ({len(filtered_list)}), ordered by vote_average:")


        sorted_df = filtered_list.sort_values(by='vote_average', ascending=False)


        recommendations = [f"{row['original_title']} - {row['vote_average']} - {row['release_date']}" for index, row in sorted_df.iterrows()]

        return recommendations
    else:
        print(f"No movies of genre {genre} found in the list, but the AI can recommend something else.")


        sorted_df = df.sort_values(by='vote_average', ascending=False)


        return sorted_df[['original_title', 'vote_average', 'release_date']]



# url = "https://api.themoviedb.org/3/discover/movie?language=en-US&page=10&region=japan&sort_by=popularity.desc&without_genres=Horror"
def userinfo():
    try:
        year=int(input("Please input the year: "))
        region = str(input("Please input the region: ").lower())
        without_genres = str(input("Please input the genre you want to EXCLUDE: "))
        language = str(input("Please input the language: ").lower())
        sort_by = input("Please input sorting method: ").lower()
        # https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
        # https://developer.themoviedb.org/docs/languages

        # year=int(input("What year? "))
        # region = input("What region? ").lower()
        # without_genres = input("Do you dislike any movie type?").lower()
        # language = input("What language? ")

        name = (str(input("Please input name: ")))
        age = float(input("Please input age: "))
        movietype = (str(input("Please input the movie genre: ")))
        typeperson= input("Please input one adjective desribing {0}: ".format(name))
    except:
        print("Input is invalid. Please restart the program.")

  # Parameters
    params = {
        "region": region,
        "without_genres": without_genres,
        "language": language,
        "year": year,
        "page": 10,  # You can change this to the desired page number
        "sort_by": sort_by,
        "name": name,
        "age": age,
        "movietype": movietype,
        "typeperson": typeperson
    }
    return params

params=userinfo()
df=collectmovies(params)
movie_recommendations=movie_list(df, "Drama")
df


print(f"-----------------------AI recommendation-----------------------")
print(f"-----------------------AI recommendation-----------------------")
print(f"-----------------------AI recommendation-----------------------")
movie_recommendations=movie_list(df, params["movietype"])

prompt = f"Recommend a good movie for {params['name']}, his age is {params['age']}, he is a {params['typeperson']} person, from this list: {movie_recommendations}"
response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=30)
ai_pick = response.choices[0].text.strip()

print(ai_pick)

Please input the year: 2018
Please input the region: us
Please input the genre you want to EXCLUDE: Action
Please input the language: en
Please input sorting method: revenue.desc
Please input name: Toshi
Please input age: 18
Please input the movie genre: Drama
Please input one adjective desribing Toshi: calm
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
AI recommendation: I would recommend 'The Guernsey Literary & Potato Peel Pie Society' for Toshi. It has a rating of 7.5,
